In [7]:
import open3d as o3d
from open3d.open3d.geometry import voxel_down_sample,estimate_normals
import numpy as np
import glob
import copy
import math

voxel_size = 0.035
max_correspondence_dist_coarse = voxel_size * 15
max_correspondence_dist_fine = voxel_size * 1.5
radius_normal = voxel_size * 2
radius_feature = voxel_size * 5
voxel_radius = [0.04, 0.02, 0.01]
max_iter = [50, 30, 14]
current_transformation = np.identity(4)

def preprocess_point_cloud(pcd, voxel_size):
    o3d.geometry.estimate_normals(pcd,o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    #radius_feature = voxel_size * 5
    pcd_fpfh = o3d.registration.compute_fpfh_feature(pcd,o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_fpfh

def load_point_clouds(directory ,n):
    pcds = []
    fpfh = []
    print("loading point cloud")
    for i in range(n):
        #print("point cloud %d" %i)
        pcd_d = o3d.io.read_point_cloud(directory + "%d.ply" % i)
        o3d.geometry.estimate_normals(pcd_d,o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
        pcds.append(pcd_d)
        pcd_fpfh = o3d.registration.compute_fpfh_feature(pcd_d,o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
        #preprocess_point_cloud(pcd_d, voxel_size)
        fpfh.append(pcd_fpfh)  
    return pcds, fpfh

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target])

def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    voxel_size = 0.035
    distance_threshold = voxel_size * 1.5
    result = o3d.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, distance_threshold,
        o3d.registration.TransformationEstimationPointToPoint(False), 30, 
        [o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.5),
         o3d.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
        o3d.registration.RANSACConvergenceCriteria(4000000, 500))
    return result

def pairwise_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    current_transformation = np.identity(4)
    for scale in range(3):
        iter = max_iter[scale]
        radius = voxel_radius[scale]
        #print([iter, radius, scale])
        
        #print("3-1. Downsample with a voxel size %.2f" % radius)
        source_down = voxel_down_sample(source, radius)
        target_down = voxel_down_sample(target, radius)

        '''
        #print("3-2. Estimate normal.")
        o3d.geometry.estimate_normals(source_down, o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
        o3d.geometry.estimate_normals(target_down, o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

        '''
        result_ransac = execute_global_registration(source, target,source_fpfh, target_fpfh, voxel_size)
        result_icp = o3d.registration.registration_colored_icp(
            source_down, target_down, radius, result_ransac.transformation,
            o3d.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                                relative_rmse=1e-6,
                                                                max_iteration=iter))
        current_transformation = result_icp.transformation
    
    #draw_registration_result(source, target,result_icp.transformation)
    information_icp = o3d.registration.get_information_matrix_from_point_clouds(source, target, 
                                                                                max_correspondence_dist_fine,
                                                                                current_transformation)
    return current_transformation, information_icp

def main_registration(pcds,fpfh, max_correspondence_dist_coarse,max_correspondence_dist_fine, voxel_size):
    pose_graph = o3d.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
            target_id = (source_id + 1) % n_pcds
            #print("performing transformation for:")
            transformation_icp, information_icp = pairwise_registration(pcds[source_id], pcds[target_id], 
                                                                        fpfh[source_id], fpfh[target_id],
                                                                        voxel_size)
            #draw_registration_result(pcds[source_id], pcds[target_id], transformation_icp)
            #print("Build o3d.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(o3d.registration.PoseGraphNode(np.linalg.inv(odometry)))
                pose_graph.edges.append(o3d.registration.PoseGraphEdge(source_id,target_id,                                                             transformation_icp, information_icp,uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(o3d.registration.PoseGraphEdge(source_id,target_id,                                                             transformation_icp, information_icp,uncertain=True))
    return pose_graph

def final_fun(dirtry):
    #Directory of Point-cloud goes here
    directory = dirtry
    n = len(glob.glob1(directory,"*.ply"))
    o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)
    
    pcds_down, pcd_fpfh = load_point_clouds(directory,n)
    #o3d.visualization.draw_geometries(pcds_down)
    n_pcds = len(pcds_down)
    
    #Implementing Full Registration
    print("Full registration ...")
    pose_graph = main_registration(pcds_down,pcd_fpfh, max_correspondence_dist_coarse,max_correspondence_dist_fine, voxel_size)

    #Implementing Global Optimization
    print("Optimizing PoseGraph ...")
    option = o3d.registration.GlobalOptimizationOption                (max_correspondence_distance=max_correspondence_dist_fine,
                                                       edge_prune_threshold=0.25,reference_node=0)
    o3d.registration.global_optimization(
        pose_graph, o3d.registration.GlobalOptimizationLevenbergMarquardt(),
        o3d.registration.GlobalOptimizationConvergenceCriteria(), option)

    #print("Transform points and display")
    pcd_combined = o3d.geometry.PointCloud()
    for point_id in range(len(pcds_down)):
        #print(point_id)
        #print(pose_graph.nodes[point_id].pose)
        pcds_down[point_id].transform(pose_graph.nodes[point_id].pose)
        pcd_combined += pcds_down[point_id]
    #o3d.visualization.draw_geometries(pcds_down)
    pcd_combined_down = o3d.geometry.voxel_down_sample(pcd_combined,voxel_size=0.001)
    #print(pcd_combined_down)
    o3d.io.write_point_cloud(dirtry + "multiway_registration_full.ply", pcd_combined)
    #o3d.io.write_point_cloud(dirtry + "multiway_registration_reduced.ply", pcd_combined_down)
    o3d.visualization.draw_geometries([pcd_combined])
    #C:/Users/Kathan/Desktop/ACLAB/3D_Registeration/DATA/code/baby/try1/test1/
    

In [8]:
import easydict
import time
if __name__ == "__main__":
    n = input("How many Pcds do you have?")
    args = easydict.EasyDict({
                           "dir": "C:/Users/Kathan/Desktop/ACLAB/3D_Registeration/DATA/code/baby/tr2/six/",
                           "n": int(n),
                         })
    start_time = time.time()        
    final_fun(args.dir)
    print("--- %s seconds ---" % (time.time() - start_time))

loading point cloud
Full registration ...
Optimizing PoseGraph ...
--- 56.91076374053955 seconds ---


In [ ]:
def preprocess_point_cloud(pcd, voxel_size):
    #print(":: Downsample with a voxel size %.3f." % voxel_size)
    #pcd_down = o3d.geometry.voxel_down_sample(pcd, voxel_size)
    radius_normal = voxel_size * 2
    #print(":: Estimate normal with search radius %.3f." % radius_normal)
    o3d.geometry.estimate_normals(pcd,o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    radius_feature = voxel_size * 5
    #print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.registration.compute_fpfh_feature(pcd,o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_fpfh


In [ ]:
def coloured_registration(source_down,target_down):
    # This is implementation of following paper
    # J. Park, Q.-Y. Zhou, V. Koltun,
    # Colored Point Cloud Registration Revisited, ICCV 2017
    
    print("3. Colored point cloud registration")
    for scale in range(3):
        iter = max_iter[scale]
        radius = voxel_radius[scale]
        print([iter, radius, scale])

        print("3-1. Downsample with a voxel size %.2f" % radius)
        source_down = source.voxel_down_sample(radius)
        target_down = target.voxel_down_sample(radius)

        print("3-2. Estimate normal.")
        source_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
        target_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

        print("3-3. Applying colored point cloud registration")
        result_icp = o3d.registration.registration_colored_icp(
            source_down, target_down, radius, current_transformation,
            o3d.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                                relative_rmse=1e-6,
                                                                max_iteration=iter))
        current_transformation = result_icp.transformation
        print(result_icp)
    draw_registration_result_original_color(source, target,
                                        result_icp.transformation)